# Random Operators: examples of random states and channels

In [ ]:
import numpy as np
from sympy.combinatorics import Permutation
import forest_benchmarking.random_operators as rand_ops
import numpy.linalg

## Complex Ginibre ensemble

In [ ]:
rand_ops.ginibre_matrix_complex(2,2)

## Haar random unitary

In [ ]:
U = rand_ops.haar_rand_unitary(2)
print(np.around(U.dot(np.transpose(np.conj(U))),decimals=15))
print(np.around(U.dot(np.transpose(np.conj(U))),decimals=16))
# only good to 16 decimal places...

## Haar random pure state 

In [ ]:
psi = rand_ops.haar_rand_state(2)
print(psi)

## Ginibre State (mixed state with rank K)

In [ ]:
# mixed single qubit state
print(np.around(rand_ops.ginibre_state_matrix(2,2),4))
print("----------------------")
# mixed two qubit state
print(np.around(rand_ops.ginibre_state_matrix(4,2),4))

In [ ]:
# you cant have Rank > Hilbert space Dim
rand_ops.ginibre_state_matrix(2,3)

## State from Bures measure

In [ ]:
rand_ops.bures_measure_state_matrix(2)

## Uniform ensemble of CPTP maps

In [ ]:
# random quantum channel on one qubit in Choi form
choi = rand_ops.rand_map_with_BCSZ_dist(2,2)
choi

In [ ]:
choi.shape

## Permutations of operators on tensor product Hilbert spaces

In [ ]:
# pick a hilbert space dimension
D = 2
# pick a way you want to permute the operators
perm =[1,2,0]
# Note: the number of elements in the permutation determines
# the number of Hilbert spaces you are considering.

^^ here the Identity permutation is P = [0,1,2] which maps (a,b,c) to (a,b,c).
The permutation P = [1,2,0] maps (a,b,c) to (b,c,a).

### Create the basis states in the Hilbert space

In [ ]:
basis = list(range(0,D))
states = []
for jdx in basis:
    emptyvec = np.zeros((D,1))
    emptyvec[jdx] =1
    states.append(emptyvec)

### Create inital state and answer after applying the permutation [1,2,0]

In [ ]:
inital_vector = np.kron(np.kron(states[0],states[0]),states[1]) # before permuting anything
perm_vector = np.kron(np.kron(states[0],states[1]),states[0]) # apply the permutation by hand

### create permutation operator

In [ ]:
P_120 = rand_ops.permute_tensor_factors(D, perm)

### check the permutation operator applied to the intial vector gives the correct answer

In [ ]:
answer = np.matmul(P_120,inital_vector)

In [ ]:
np.matmul(perm_vector.T,answer)